<a href="https://colab.research.google.com/github/machine-learning-ss/data-science/blob/master/001_predicting_telco_customer_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting Telco Customer Churn**

Customer churn is a major problem and one of the most important concerns for large companies. Due to the direct effect on the revenues of the companies, especially in the telecom field, companies are seeking to develop means to predict potential customer to churn. Therefore, finding factors that increase customer churn is important to take necessary actions to reduce this churn.

The main contribution of our work is to develop a churn prediction model which assists telecom operators to predict customers who are most likely subject to churn. The model developed in this work uses machine learning techniques on big data platform and builds a new way of features’ engineering and selection.

Here we model the classification model from telco customer churn data. This data is consist of customer profile, customer subscription history, and their churn information. We will predict customer behavior to retain customers. You can analyze all relevant customer data and develop focused customer retention programs

Each row represents a customer, each column contains customer’s attributes described below:
1.   customerID : Customer ID
2.   gender : Whether the customer is a male or a female
3.   SeniorCitizen : Whether the customer is a senior citizen or not (1, 0)
4.   Partner : Whether the customer has a partner or not (Yes, No)
5.   Dependents : Whether the customer has dependents or not (Yes, No)
6.   tenure : Number of months the customer has stayed with the company
7.   PhoneService : Whether the customer has a phone service or not (Yes, No)
8.   MultipleLines : Whether the customer has multiple lines or not (Yes, No, No phone service)
9.   InternetService : Customer’s internet service provider (DSL, Fiber optic, No)
10.   OnlineSecurity: Whether the customer has online security or not (Yes, No, No internet service)
11.   OnlineBackup: Whether the customer has online backup or not (Yes, No, No internet service)
12.   DeviceProtection: Whether the customer has device protection or not (Yes, No, No internet service)
13.   TechSupport : Whether the customer has tech support or not (Yes, No, No internet service)
14.   StreamingTV : Whether the customer has streaming TV or not (Yes, No, No internet service)
15.   StreamingMovies : Whether the customer has streaming movies or not (Yes, No, No internet service)
16.   Contract : The contract term of the customer (Month-to-month, One year, Two year)
17.   PaperlessBilling : Whether the customer has paperless billing or not (Yes, No)
18.   PaymentMethod : The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
19.   MonthlyCharges : The amount charged to the customer monthly
20.   TotalCharges : The total amount charged to the customer
21.   Churn Whether: the customer churned or not (Yes or No)

Source: https://www.kaggle.com/blastchar/telco-customer-churn, https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0191-6

We use classification model to predict telco customer churn. Classification is the problem of identifying to which of a set of categories (sub-populations) a new observation belongs, on the basis of a training set of data containing observations (or instances) whose category membership is known.

#### **Install and Import Libraries**

Before we begin to implement our classifier, we need to import some libraries to use them later. Here are the libraries we need to import.

***Install Library***

In [ ]:
# Install Category Encoders
! pip install category_encoders

***Import Libraries***

In [ ]:
# Import Library for Data Manipulation
import pandas as pd
import category_encoders as ce

# Import Library for Machine Learning
import sklearn.metrics as metrics

# Import Library for Visualization
import matplotlib. pyplot as plt
import seaborn as sns

#### **Import Dataset**

Import our customer churn dataset into this notebook using Pandas library. Then, we discover the dataset information and statistics.

***Customer Churn Data***

In [ ]:
# Import Dataset
df_churn = pd.read_csv('https://raw.githubusercontent.com/machine-learning-ss/dataset/master/telco_customer_churn_dataset.csv', sep =',')
df_churn

In [ ]:
# Prints the Dataset Information
df_churn.info()

In [ ]:
# Prints Descriptive Statistics
df_churn.describe().transpose()

#### **Explore the Dataset**

We need to visualize the data before implement our classifier. Data visualization is the act of taking information (data) and placing it into a visual context, such as a map or graph. Data visualizations make big and small data easier for the human brain to understand, and visualization also makes it easier to detect patterns, trends, and outliers in groups of data. Here we use Seaborn library.

***Visualize Correlation between Features***

In [ ]:
# Draw Correlation Map
sns.clustermap(df_churn.corr(), center=0, cmap='vlag', linewidths=.75)

#### **Preprocess the Data**

We should transforms raw data into an understandable format. Raw data cannot be sent through a model because would cause certain errors. That is why we need to preprocess data before sending through a model.

***Handling Missing Values***

In [ ]:
# Check for Missing Values
df_churn.isnull().sum()

In [ ]:
# Search for Median Value
median = df_churn['TotalCharges'].median()

# Use Median to Replace Missing Values
df_churn['TotalCharges'].fillna(median, inplace=True)

# Check for Missing Values
df_churn.isnull().sum()

***Encode Categorical Data***

Data encoding purposed to transform a categorical data into binary numeric format. Here we use OneHotencoder module from sklearn to encode our categorical data.

In [ ]:
# Import Module
from sklearn.preprocessing import OneHotEncoder

# Encoder
encoder = OneHotEncoder(sparse=False)

# Encode Categorical Data
df_churn2 = pd.DataFrame(encoder.fit_transform(df_churn[['gender', 'InternetService', 'Contract', 'PaymentMethod']]))
df_churn2.columns = encoder.get_feature_names(['gender', 'InternetService', 'Contract', 'PaymentMethod'])

# Replace Categotical Data with Encoded Data
df_churn_encoded = df_churn.drop(['gender', 'InternetService', 'Contract', 'PaymentMethod'] ,axis=1, inplace=True)
df_churn_encoded = pd.concat([df_churn, df_churn2], axis=1)

# Show Encoded Dataframe
df_churn_encoded

***Select Feature and Target***

Features are individual independent variables that act as the input in your system while target is whatever the output of the input variables. 

In [ ]:
# Select Features
feature = df_churn_encoded.drop(['customerID', 'Churn'], axis=1)
feature

In [ ]:
# Select Target
target = df_churn_encoded['Churn']
target

***Set Training and Testing Data***

The next step is to split our data into tran and test sets. For this purpose, we use the scikit-learn's train_test_split function.

In [ ]:
# Import Module
from sklearn.model_selection import train_test_split, cross_val_score

# Set Training and Testing Data (70:30)
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(feature , target, shuffle = True, test_size=0.3, random_state=1)

# Show the Training and Testing Data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

#### **Modeling**

##### **Decision Tree Classifier**

Decision tree learning is one of the predictive modelling approaches used in statistics, data mining and machine learning. It uses a decision tree (as a predictive model) to go from observations about an item (represented in the branches) to conclusions about the item's target value (represented in the leaves).

***Build Model***

In [ ]:
# Import library
from sklearn import tree

# Modeling Decision Tree
dtc= tree.DecisionTreeClassifier(min_impurity_decrease=0.01)
dtc.fit(X_train, y_train)

# Predict Test Data 
dtc_pred = dtc.predict(X_test)
dtc_pred

In [ ]:
# Visualize Tree

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

feature_names = feature.columns[0:]

dot_data = StringIO()
export_graphviz(dtc, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                class_names=['notchurn', 'churn'],
                feature_names=feature_names)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

***Model Evaluation***

In [ ]:
# Confsion Matrix
cm_dtc = metrics.confusion_matrix(y_test, dtc_pred)
cm_dtc

In [ ]:
# Accuracy, Precision, Recall
acc_dtc = metrics.accuracy_score(y_test, dtc_pred)
prec_dtc = metrics.precision_score(y_test, dtc_pred)
rec_dtc = metrics.recall_score(y_test, dtc_pred)
f1_dtc = metrics.f1_score(y_test, dtc_pred)
kappa_dtc = metrics.cohen_kappa_score(y_test, dtc_pred)

# Show Accuracy, Precision, Recall
print('Accuracy:', acc_dtc)
print('Precision:', prec_dtc)
print('Recall:', rec_dtc)
print('F1 Score:', f1_dtc)
print('Cohens Kappa Score:', kappa_dtc)

In [ ]:
# Import Visualization Package
plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('ggplot')

# Visualize ROC Curve
dtc_prob = dtc.predict_proba(X_test)[::,1]
fp_rate_dtc, tp_rate_dtc, _ = metrics.roc_curve(y_test,  dtc_prob)
auc_dtc = metrics.roc_auc_score(y_test, dtc_prob)
plt.plot(fp_rate_dtc, tp_rate_dtc, label='Decision Tree, auc='+str(auc_dtc))
plt.xlabel('false positive rate') 
plt.ylabel('true positive rate')
plt.legend(loc=4)
plt.show()

##### **Naive Bayes Classifier**

Naïve Bayes classifiers are a family of simple "probabilistic classifiers" based on applying Bayes' theorem with strong (naïve) independence assumptions between the features. Naive Bayes is a simple technique for constructing classifiers: models that assign class labels to problem instances, represented as vectors of feature values, where the class labels are drawn from some finite set.

***Build Model***

In [ ]:
# Import Module
from sklearn.naive_bayes import GaussianNB 

# Modeling Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)

# Predict Test Data 
nb_pred = nb.predict(X_test)
nb_pred

***Model Evaluation***

In [ ]:
# Confsion Matrix
cm_nb = metrics.confusion_matrix(y_test, nb_pred)
cm_nb

In [ ]:
# Accuracy, Precision, Recall
acc_nb = metrics.accuracy_score(y_test, nb_pred)
prec_nb = metrics.precision_score(y_test, nb_pred)
rec_nb = metrics.recall_score(y_test, nb_pred)
f1_nb = metrics.f1_score(y_test, nb_pred)
kappa_nb = metrics.cohen_kappa_score(y_test, nb_pred)

# Show Accuracy, Precision, Recall
print('Accuracy:', acc_nb)
print('Precision:', prec_nb)
print('Recall:', rec_nb)
print('F1 Score:', f1_nb)
print('Cohens Kappa Score:', kappa_nb)

In [ ]:
# Import Visualization Package
plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('ggplot')

# Visualize ROC Curve
nb_prob = nb.predict_proba(X_test)[::,1]
fp_rate_nb, tp_rate_nb, _ = metrics.roc_curve(y_test,  nb_prob)
auc_nb = metrics.roc_auc_score(y_test, nb_prob)
plt.plot(fp_rate_nb, tp_rate_nb, label='Naive Bayes, auc='+str(auc_nb))
plt.xlabel('false positive rate') 
plt.ylabel('true positive rate')
plt.legend(loc=4)
plt.show()

#### **Evaluating Models**

***Compare Model Performance***

In [ ]:
# Comparing Model Performance
print('Decision Tree Accuracy =',acc_dtc)
print('Decision Tree Precision =',prec_dtc)
print('Decision Tree Recall =',rec_dtc)
print('Decision Tree F1-Score =', f1_dtc)

print('_______________________')

print('Naive Bayes Accuracy =', acc_nb)
print('Naive Bayes Precision =', prec_nb)
print('Naive Bayes Recall =', rec_nb)
print('Naive Bayes F1-Score =', f1_nb)

***Compare ROC Curve***

In [ ]:
# Comparing ROC Curve
plt.plot(fp_rate_dtc,tp_rate_dtc,label='Decision Tree, auc='+str(auc_dtc))
plt.plot(fp_rate_nb,tp_rate_nb,label='Naive Bayes, auc='+str(auc_nb))
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc=4)
plt.show()

#### **Predict New Data**

***Import New Data***

In [ ]:
# Import New Dataset
df_new_churn = pd.read_csv('https://raw.githubusercontent.com/machine-learning-ss/dataset/master/telco_customer_churn_new.csv', sep =',')
df_new_churn

***Preprocess the New Data***

In [ ]:
# Import Module
from sklearn.preprocessing import OneHotEncoder

# Encoder
encoder = OneHotEncoder(sparse=False)

# Encode Categorical Data
df_new_churn2 = pd.DataFrame(encoder.fit_transform(df_new_churn[['gender', 'InternetService', 'Contract', 'PaymentMethod']]))
df_new_churn2.columns = encoder.get_feature_names(['gender', 'InternetService', 'Contract', 'PaymentMethod'])

# Concat the Encoded Data
df_new_churn_encoded = df_new_churn.drop(['gender', 'InternetService', 'Contract', 'PaymentMethod'] ,axis=1, inplace=True)
df_new_churn_encoded = pd.concat([df_new_churn, df_new_churn2], axis=1)

# Show Encoded Dataframe
df_new_churn_encoded

In [ ]:
# Select Features
new_feature = df_new_churn_encoded.drop(['customerID'], axis=1)
new_feature

***Predict New Customer Data***

In [ ]:
# Predict using Decision Tree Classifier
dtc_prediction = pd.DataFrame(dtc.predict(new_feature), columns = ['dtc_prediction'])
dtc_prediction.reset_index()
dtc_prediction

In [ ]:
# Predict using Naive Bayes Classifier
nb_predicton = pd.DataFrame(nb.predict(new_feature), columns = ['nb_prediction'])
nb_predicton.reset_index()
nb_predicton

***Show Prediction Comparation***

In [ ]:
# Show Prediction Result
churn_prediction = pd.concat([df_new_churn, dtc_prediction, nb_predicton], axis=1)
churn_prediction

***Save Prediction Result***

In [33]:
# Save Prediction Result
churn_prediction.to_csv('churn_prediction.csv', index=False)